<a href="https://colab.research.google.com/github/codehardworks/cryptotrading/blob/main/BNB_interactivo_v3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Cálculo Interactivo de Niveles de Trading para Crypto/USDT

Este notebook te permitirá **ingresar tus datos** (MA7, MA25, MA99, precio actual y máximo reciente)
y calcular automáticamente las zonas de entrada, stop loss y take profits.


In [ ]:
"""Herramientas de cálculo para planificación de TPs/SL y OCOs.

- Organizado y deduplicado.
- Docstrings en español.
- Tipado y constantes agrupadas.
- API original preservada (nombres y firmas).
 @Tami, @Juancho
"""
from dataclasses import dataclass
from typing import Dict, Tuple, Any
import math
import pandas as pd

# =============================================================================
# Constantes
# =============================================================================
FEE_SIDE: float = 0.00075  # 0,075% por lado (fijo)

# =============================================================================
# Utilidades
# =============================================================================

def p2f(pct_val: float) -> float:
    """Convierte porcentaje (p.ej. 1.5) a fracción (0.015)."""
    return float(pct_val) / 100.0


def f2p(frac: float) -> float:
    """Convierte fracción (0.015) a porcentaje (1.5)."""
    return frac * 100.0

# =============================================================================
# Niveles de entrada/SL/TP a partir de MA25 y máximo reciente
# =============================================================================

@dataclass
class Niveles:
    entrada_baja: float
    entrada_alta: float
    stop_loss: float
    take_profit_1: float
    take_profit_2: float


def calcular_niveles(
    ma25: float,
    max_reciente: float,
    margen_bajo_pct: float = 0.5,
    margen_alto_pct: float = 0.2,
    stop_bajo_pct: float = 1.5,
    tp2_pct: float = 3.0,
) -> Niveles:
    """Calcula niveles a partir de MA25 y máximo reciente.

    Args:
        ma25: Media móvil 25.
        max_reciente: Máximo reciente para TP1.
        margen_bajo_pct: % por debajo de MA para entrada baja.
        margen_alto_pct: % por encima de MA para entrada alta.
        stop_bajo_pct: % por debajo de MA para SL.
        tp2_pct: % adicional a partir de TP1 para TP2.

    Returns:
        Instancia de `Niveles` con los cinco precios.
    """
    entrada_baja = ma25 * (1 - margen_bajo_pct / 100.0)
    entrada_alta = ma25 * (1 + margen_alto_pct / 100.0)
    stop_loss = ma25 * (1 - stop_bajo_pct / 100.0)
    take_profit_1 = max_reciente
    take_profit_2 = take_profit_1 * (1 + tp2_pct / 100.0)
    return Niveles(entrada_baja, entrada_alta, stop_loss, take_profit_1, take_profit_2)

# =============================================================================
# Cálculos básicos de PnL y R múltiple (posición long)
# =============================================================================

def tp_minimo_para_rentabilidad(
    entry_price: float, fee_side: float = FEE_SIDE
) -> Tuple[float, float]:
    """Precio/porcentaje mínimo de TP para no perder dinero neto de comisiones.

    Fórmula: neto >= 0 con fee por lado `fee_side`.
    net = q*((1-f)*x - (1+f)*e) con q>0. Para net>=0 => x >= e*(1+f)/(1-f)

    Returns:
        (exit_price_min, tp_min_pct)
    """
    exit_price_min = entry_price * (1 + fee_side) / (1 - fee_side)
    tp_min_frac = (exit_price_min / entry_price) - 1.0
    return float(exit_price_min), float(f2p(tp_min_frac))


def pnl_neto_long(
    entry: float, exit_price: float, qty: float, fee_side: float = FEE_SIDE
) -> float:
    """PnL neto (incluye fee por lado) para una posición long."""
    return (exit_price - entry) * qty - fee_side * (entry * qty + exit_price * qty)


def r_multiple_neto(
    entry: float,
    qty: float,
    tp_pct: float,
    sl_pct: float,
    fee_side: float = FEE_SIDE,
) -> Tuple[float, float, float, float, float]:
    """Calcula R múltiple neto vs. un TP y SL (%), y devuelve métricas útiles.

    Returns:
        (R, reward, risk, exit_tp, exit_sl)
    """
    tp_f = p2f(tp_pct)
    sl_f = p2f(sl_pct)
    exit_tp = entry * (1 + tp_f)
    exit_sl = entry * (1 - sl_f)
    reward = pnl_neto_long(entry, exit_tp, qty, fee_side)
    risk = abs(pnl_neto_long(entry, exit_sl, qty, fee_side))
    return (reward / risk) if risk > 0 else math.inf, reward, risk, exit_tp, exit_sl


# =============================================================================
# Simulaciones tabulares (útiles para análisis)
# =============================================================================

def simular_tp(
    entry: float,
    qty: float,
    max_pct: float = 10.0,
    paso: float = 0.25,
    fee_side: float = FEE_SIDE,
    sl_pct_ref: float | None = None,
) -> pd.DataFrame:
    """Tabla de PnL neto al variar TP desde 0 hasta `max_pct`.

    Si `sl_pct_ref` se especifica, añade R neto vs. ese SL de referencia.
    """
    filas = []
    p = 0.0
    while p <= max_pct + 1e-9:
        exit_p = entry * (1 + p2f(p))
        pnl = pnl_neto_long(entry, exit_p, qty, fee_side)
        R = None
        if sl_pct_ref is not None:
            R, *_ = r_multiple_neto(entry, qty, p, sl_pct_ref, fee_side)
        filas.append(
            {
                "TP_%": round(p, 4),
                "ExitPrice": round(exit_p, 8),
                "PnL_neto": round(pnl, 8),
                "R_neto_vs_SLref": None if R is None else round(R, 6),
            }
        )
        p += paso
    return pd.DataFrame(filas)



def simular_sl(
    entry: float,
    qty: float,
    max_pct: float = 10.0,
    paso: float = 0.25,
    fee_side: float = FEE_SIDE,
    tp_pct_ref: float | None = None,
) -> pd.DataFrame:
    """Tabla de PnL neto al variar SL desde 0 hasta `max_pct`.

    Si `tp_pct_ref` se especifica, añade R neto vs. ese TP de referencia.
    """
    filas = []
    p = 0.0
    while p <= max_pct + 1e-9:
        exit_p = entry * (1 - p2f(p))
        pnl = pnl_neto_long(entry, exit_p, qty, fee_side)
        R = None
        if tp_pct_ref is not None:
            R, *_ = r_multiple_neto(entry, qty, tp_pct_ref, p, fee_side)
        filas.append(
            {
                "SL_%": round(p, 4),
                "StopPrice": round(exit_p, 8),
                "PnL_neto": round(pnl, 8),
                "R_neto_vs_TPref": None if R is None else round(R, 6),
            }
        )
        p += paso
    return pd.DataFrame(filas)


# =============================================================================
# Utilidades adicionales (objetivos de ganancia/pérdida neta)
# =============================================================================

def tp_minimo_para_ganar_monto(
    entry: float, qty: float, ganancia_neta: float, fee_side: float = FEE_SIDE
) -> Tuple[float, float]:
    """(exit_price_min, tp_pct_min) para lograr al menos `ganancia_neta` (>=0).

    Fórmula: q*((1-f)*x - (1+f)*e) >= G  => x >= ((G/q) + (1+f)*e)/(1-f)
    """
    if qty <= 0:
        raise ValueError("qty debe ser > 0")
    if ganancia_neta < 0:
        raise ValueError("ganancia_neta debe ser >= 0")

    x_min = ((ganancia_neta / qty) + (1 + fee_side) * entry) / (1 - fee_side)
    tp_frac = (x_min / entry) - 1.0
    return round(x_min, 8), round(f2p(tp_frac), 8)


def sl_maximo_para_perder_monto(
    entry: float, qty: float, perdida_neta: float, fee_side: float = FEE_SIDE
) -> Tuple[float, float]:
    """(stop_price_max, sl_pct_max) para realizar `perdida_neta` (>=0) neta de fees.

    net = q*((1-f)*x - (1+f)*e) = -L  => x = ((1+f)*e - (L/q)) / (1-f)

    Casos:
      - Si L <= costo por comisiones al cerrar en el mismo precio (2*f*e*q),
        el movimiento requerido es 0% (stop = entry, sl%=0).
      - Si L > pérdida máxima posible (q*(1+f)*e), es imposible.
    """
    if qty <= 0:
        raise ValueError("qty debe ser > 0")
    if perdida_neta < 0:
        raise ValueError("perdida_neta debe ser >= 0")

    loss_by_fees = 2 * fee_side * entry * qty
    max_loss_possible = (1 + fee_side) * entry * qty  # cuando x -> 0

    if perdida_neta <= loss_by_fees:
        return round(entry, 8), 0.0  # ya se pierde eso o más solo por comisiones

    if perdida_neta > max_loss_possible:
        raise ValueError(
            "La pérdida solicitada supera la pérdida máxima posible con posición long."
        )

    x = ((1 + fee_side) * entry - (perdida_neta / qty)) / (1 - fee_side)
    sl_frac = max(0.0, 1.0 - (x / entry))
    return round(x, 8), round(f2p(sl_frac), 8)


# Alias por compatibilidad

def tp_maximo_para_perder_monto(
    entry: float, qty: float, perdida_neta: float, fee_side: float = FEE_SIDE
) -> Tuple[float, float]:
    """Alias de `sl_maximo_para_perder_monto` (para 'perder' hablamos de SL)."""
    return sl_maximo_para_perder_monto(entry, qty, perdida_neta, fee_side)


# =============================================================================
# OCO Planner (2 OCOs)
# =============================================================================

@dataclass
class OCOParams:
    tp_pct: float
    sl_pct: float
    qty: float


@dataclass
class ReconfParams:
    lock_profit_after_tp1_pct: float  # mover SL2 por encima de entry (%). 0=BE
    tp2_retarget_pct: float  # nuevo TP2 desde entry (%)


def plan_dos_OCOs(
    entry: float,
    qty_total: float,
    fee_side: float,
    sl_pct_global: float,
    split_qty_pct_for_oco1: float,
    oco1_tp_pct: float,
    oco2_tp_pct: float,
    reconf: ReconfParams,
) -> Dict[str, Any]:
    """Calcula PnL por escenarios para dos OCOs más reconfiguración tras TP1."""
    sl_f = p2f(sl_pct_global)
    oco1_qty = qty_total * (split_qty_pct_for_oco1 / 100.0)
    oco2_qty = qty_total - oco1_qty

    oco1 = OCOParams(tp_pct=oco1_tp_pct, sl_pct=sl_pct_global, qty=oco1_qty)
    oco2 = OCOParams(tp_pct=oco2_tp_pct, sl_pct=sl_pct_global, qty=oco2_qty)

    oco1_tp_price = entry * (1 + p2f(oco1.tp_pct))
    oco2_tp_price = entry * (1 + p2f(oco2.tp_pct))
    sl_price = entry * (1 - sl_f)

    pnl_oco1_tp = pnl_neto_long(entry, oco1_tp_price, oco1.qty, fee_side)
    pnl_oco1_sl = pnl_neto_long(entry, sl_price, oco1.qty, fee_side)
    pnl_oco2_tp = pnl_neto_long(entry, oco2_tp_price, oco2.qty, fee_side)
    pnl_oco2_sl = pnl_neto_long(entry, sl_price, oco2.qty, fee_side)

    # RECONFIG tras TP1
    sl2_new = entry * (1 + p2f(reconf.lock_profit_after_tp1_pct))
    tp2_new = entry * (1 + p2f(reconf.tp2_retarget_pct))

    pnl_oco2_after_tp1_to_newTP = pnl_neto_long(entry, tp2_new, oco2.qty, fee_side)
    pnl_oco2_after_tp1_to_newSL = pnl_neto_long(entry, sl2_new, oco2.qty, fee_side)

    escenarios = {
        "Solo_SL_global": round(pnl_oco1_sl + pnl_oco2_sl, 8),
        "TP1_y_luego_TP2_reconfig": round(pnl_oco1_tp + pnl_oco2_after_tp1_to_newTP, 8),
        "TP1_y_luego_SL2_reconfig": round(pnl_oco1_tp + pnl_oco2_after_tp1_to_newSL, 8),
        "TP2_sin_TP1": round(pnl_oco2_tp + pnl_oco1_sl, 8),
        "TP1_sin_TP2": round(pnl_oco1_tp + pnl_oco2_sl, 8),
    }

    return {
        "entrada": entry,
        "sl_price": round(sl_price, 8),
        "oco1": {
            "qty": round(oco1.qty, 8),
            "tp_%": oco1.tp_pct,
            "tp_price": round(oco1_tp_price, 8),
            "sl_%": sl_pct_global,
            "sl_price": round(sl_price, 8),
        },
        "oco2": {
            "qty": round(oco2.qty, 8),
            "tp_%": oco2.tp_pct,
            "tp_price": round(oco2_tp_price, 8),
            "sl_%": sl_pct_global,
            "sl_price": round(sl_price, 8),
        },
        "reconfiguracion_si_TP1": {
            "nuevo_SL2_%_sobre_entry": reconf.lock_profit_after_tp1_pct,
            "nuevo_SL2_price": round(sl2_new, 8),
            "nuevo_TP2_%_sobre_entry": reconf.tp2_retarget_pct,
            "nuevo_TP2_price": round(tp2_new, 8),
        },
        "pnl_escenarios": escenarios,
    }


__all__ = [
    # Niveles
    "Niveles",
    "calcular_niveles",
    # Utilidades básicas
    "FEE_SIDE",
    "p2f",
    "f2p",
    "tp_minimo_para_rentabilidad",
    "pnl_neto_long",
    "r_multiple_neto",
    "simular_tp",
    "simular_sl",
    # Utilidades avanzadas
    "tp_minimo_para_ganar_monto",
    "sl_maximo_para_perder_monto",
    "tp_maximo_para_perder_monto",
    # OCO
    "OCOParams",
    "ReconfParams",
    "plan_dos_OCOs",
]


In [ ]:
# ==============================
# Helpers de entrada / formato
# ==============================

from typing import Optional  # <-- AÑADIDO: era usado pero no importado

LABEL_W = 34
LINE_W  = 74

def _ask_float(msg: str, default: Optional[float] = None) -> float:
    """Pide un float al usuario; acepta coma o punto. Usa default si se deja vacío."""
    tip = f" [{default}]" if default is not None else ""
    while True:
        s = input(f"{msg}{tip}: ").strip().replace(",", ".")
        if s == "" and default is not None:
            return float(default)
        try:
            return float(s)
        except ValueError:
            print("Valor inválido. Intenta de nuevo.")


def _ask_yes_no(msg: str, default: bool = True) -> bool:
    """Pregunta sí/no. Enter usa default."""
    d = "S/n" if default else "s/N"
    while True:
        s = input(f"{msg} ({d}): ").strip().lower()
        if s == "":
            return default
        if s in {"s", "si", "sí", "y", "yes"}:
            return True
        if s in {"n", "no"}:
            return False
        print("Responde con 's' o 'n'.")


def _fmt(n: float, dec: int = 2) -> str:
    return f"{n:,.{dec}f}".replace(",", "_").replace(".", ",").replace("_", ".")


# —— Helpers de impresión estética ——

def print_title(text: str) -> None:
    print("\n" + "=" * LINE_W)
    print(f"{text:^{LINE_W}}")
    print("=" * LINE_W)


def print_section(text: str) -> None:
    print("\n" + "-" * LINE_W)
    print(f"{text:^{LINE_W}}")
    print("-" * LINE_W)


def print_kv(label: str, value: str | float, dec: int | None = None) -> None:
    if isinstance(value, float) and dec is not None:
        v = _fmt(value, dec)
    else:
        v = str(value)
    print(f"  {label:<{LABEL_W}} {v:>20}")

# ==============================
# Bloque 1: Niveles por MAs (conserva tus cambios)
# ==============================

def flujo_niveles() -> dict:
    print_title("Niveles desde Medias Móviles")

    ma7           = _ask_float("MA(7)")
    ma25          = _ask_float("MA(25)")
    ma99          = _ask_float("MA(99)")
    precio_actual = _ask_float("Precio actual")
    max_reciente  = _ask_float("Máximo reciente (Ultimas 24h 4366.46)", 4366.46)

    # Márgenes configurables
    margen_bajo_pct  = _ask_float("Margen bajo % debajo de MA25 para entrada baja", 0.4)
    margen_alto_pct  = _ask_float("Margen alto % arriba de MA25 para entrada alta", 0.05)
    stop_bajo_pct    = _ask_float("Stop % debajo de MA25", 1.4)
    tp2_pct          = _ask_float("TP2 % adicional sobre TP1", 3.0)

    niveles = calcular_niveles(
        ma25=ma25,
        max_reciente=max_reciente,
        margen_bajo_pct=margen_bajo_pct,
        margen_alto_pct=margen_alto_pct,
        stop_bajo_pct=stop_bajo_pct,
        tp2_pct=tp2_pct,
    )

    print_section("Parámetros")
    print_kv("MA(7)", ma7, 2)
    print_kv("MA(25)", ma25, 2)
    print_kv("MA(99)", ma99, 2)
    print_kv("Precio actual", precio_actual, 2)
    print_kv("Máximo reciente (TP1 base)", max_reciente, 2)

    print_section("Niveles sugeridos")
    print_kv("Entrada (baja)", niveles.entrada_baja, 2)
    print_kv("Entrada (alta)", niveles.entrada_alta, 2)
    print_kv("Stop Loss",       niveles.stop_loss, 2)
    print_kv("Take Profit 1",   niveles.take_profit_1, 2)
    print_kv("Take Profit 2",   niveles.take_profit_2, 2)

    EntradaAlta = niveles.entrada_alta

    # Info rápida de estado respecto al precio actual
    print_section("Estado del precio actual vs rango de entrada")
    if niveles.entrada_baja <= precio_actual <= niveles.entrada_alta:
        print("  • El precio actual está DENTRO del rango de entrada.")
    elif precio_actual < niveles.entrada_baja:
        print("  • El precio actual está por DEBAJO del rango de entrada.")
    else:
        print("  • El precio actual está por ENCIMA del rango de entrada.")

    return {
        "ma7": ma7,
        "ma25": ma25,
        "ma99": ma99,
        "precio_actual": precio_actual,
        "max_reciente": max_reciente,
        "niveles": niveles,
        "margen_bajo_pct": margen_bajo_pct,
        "margen_alto_pct": margen_alto_pct,
        "stop_bajo_pct": stop_bajo_pct,
        "tp2_pct": tp2_pct,
        "EntradaAlta": EntradaAlta,
    }

# ======================================
# Bloque 2A: Objetivos de PnL (G/L por monto con USDT→qty)
# ======================================

def flujo_objetivos(entry_default: Optional[float] = None) -> None:
    print_title("Objetivos de PnL (TP mínimo / SL máximo)")

    entry = _ask_float("Precio de entrada", entry_default if entry_default else 4190)
    while entry <= 0:
        print("El precio de entrada debe ser > 0.")
        entry = _ask_float("Precio de entrada", 100.0)

    usdt = _ask_float("Monto invertido (USDT)", 621.2)
    while usdt < 0:
        print("El monto en USDT no puede ser negativo.")
        usdt = _ask_float("Monto invertido (USDT)", 621.2)

    qty = usdt / entry
    print_section("Cantidad calculada")
    print_kv("qty = USDT / entry", qty, 8)

    G = _ask_float("Ganancia neta deseada (monto)", 10.0)
    while G < 0:
        print("La ganancia neta debe ser >= 0.")
        G = _ask_float("Ganancia neta deseada (monto)", 10.0)

    L = _ask_float("Pérdida neta máxima (monto)", 7.0)
    while L < 0:
        print("La pérdida neta debe ser >= 0.")
        L = _ask_float("Pérdida neta máxima (monto)", 7.0)

    # Cálculos
    x_min, tp_min_pct = tp_minimo_para_ganar_monto(entry, qty, ganancia_neta=G)
    print_section("Resultados TP/SL y Break-even")
    print_kv("TP mínimo (precio)", x_min, 8)
    print_kv("TP mínimo (%)", tp_min_pct, 6)

    try:
        stop_max, sl_max_pct = sl_maximo_para_perder_monto(entry, qty, perdida_neta=L)
        print_kv("SL máximo (precio)", stop_max, 8)
        print_kv("SL máximo (%)", sl_max_pct, 6)
    except ValueError as e:
        print_kv("SL máximo", f"Error: {e}")

    be_price, be_pct = tp_minimo_para_rentabilidad(entry)
    print_kv(
        f"Break-even precio (fee {FEE_SIDE*100:.3f}% lado)", be_price, 8
    )
    print_kv("Break-even TP_%", be_pct, 6)


# ======================================
# Bloque 2B: Simulaciones TP/SL (opcional)
# ======================================

def flujo_simulaciones(entry_default: Optional[float] = None) -> None:
    print_title("Simulaciones de TP/SL")

    entry_price = _ask_float(
        "Precio de entrada para simulaciones",
        entry_default if entry_default else 4190,
    )
    while entry_price <= 0:
        print("El precio de entrada debe ser > 0.")
        entry_price = _ask_float("Precio de entrada para simulaciones", 100.0)

    qty = _ask_float("Cantidad comprada (ej: 0.5)", 0.5)
    while qty < 0:
        print("La cantidad no puede ser negativa.")
        qty = _ask_float("Cantidad comprada (ej: 0.5)", 0.5)

    fee_side = FEE_SIDE
    print_section("Comisiones y referencias")
    print_kv("Comisión por lado (%)", f"{fee_side*100:.3f}")

    sl_pct_ref = _ask_float("SL de referencia para simulaciones TP (%) (ej: 2)", 2.0)
    while sl_pct_ref < 0:
        print("SL de referencia debe ser >= 0.")
        sl_pct_ref = _ask_float("SL de referencia para simulaciones TP (%) (ej: 2)", 2.0)

    tp_pct_ref = _ask_float("TP de referencia para simulaciones SL (%) (ej: 3)", 3.0)

    # Cálculos generales (siempre disponibles aunque no se simule)
    print_section("Cálculos generales")
    x_min, tpmin_pct = tp_minimo_para_rentabilidad(entry_price, fee_side)
    print_kv("TP mínimo rentable (precio)", x_min, 8)
    print_kv("TP mínimo rentable (%)", tpmin_pct, 6)

    R, reward, risk, exit_tp, exit_sl = r_multiple_neto(entry_price, qty, tp_pct_ref, sl_pct_ref, fee_side)
    print_kv("TP ref (precio)", exit_tp, 8)
    print_kv("SL ref (precio)", exit_sl, 8)
    print_kv("R múltiplo neto", f"R={R:.4f} | reward={reward:.8f} | risk={risk:.8f}")

    # Confirmación antes de simular
    if not _ask_yes_no("\n¿Quieres ejecutar las simulaciones TP/SL (tablas)?", True):
        print("\nSimulaciones omitidas por elección del usuario.")
        return

    max_tp_sim = _ask_float("Simulación TP hasta (%) (ej: 10)", 10.0)
    while max_tp_sim < 0:
        print("Debe ser >= 0.")
        max_tp_sim = _ask_float("Simulación TP hasta (%) (ej: 10)", 10.0)

    max_sl_sim = _ask_float("Simulación SL hasta (%) (ej: 10)", 10.0)
    while max_sl_sim < 0:
        print("Debe ser >= 0.")
        max_sl_sim = _ask_float("Simulación SL hasta (%) (ej: 10)", 10.0)

    paso_sim = _ask_float("Paso de simulación (%) (ej: 0.25)", 0.25)
    while paso_sim < 0.01:
        print("El paso mínimo recomendado es 0.01%.")
        paso_sim = _ask_float("Paso de simulación (%) (ej: 0.25)", 0.25)

    # Tablas de simulación
    print_section("Simulación TP (primeras filas)")
    df_tp = simular_tp(entry_price, qty, max_pct=max_tp_sim, paso=paso_sim, fee_side=fee_side, sl_pct_ref=sl_pct_ref)
    print(df_tp.head(10).to_string(index=False))

    print_section("Simulación SL (primeras filas)")
    df_sl = simular_sl(entry_price, qty, max_pct=max_sl_sim, paso=paso_sim, fee_side=fee_side, tp_pct_ref=tp_pct_ref)
    print(df_sl.head(10).to_string(index=False))


# ======================================
# Extra: evaluación R usando TP1/SL de niveles
# ======================================
def evaluar_R(entry: float, tp1_price: float, sl_price: float, qty: float) -> None:
    """Muestra R múltiple neto con TP1/SL dados desde un entry."""
    tp_pct = (tp1_price / entry - 1.0) * 100.0
    sl_pct = (1.0 - sl_price / entry) * 100.0
    R, reward, risk, exit_tp, exit_sl = r_multiple_neto(entry, qty, tp_pct, sl_pct, FEE_SIDE)

    print_title("Métrica R con TP1/SL de tus niveles")
    print_kv("TP_%", float(f"{tp_pct:.6f}"))
    print_kv("SL_%", float(f"{sl_pct:.6f}"))
    print_kv("R_neto", f"{R:.6f}")

# ======================================
# NUEVO: Planificador OCO (2 OCOs + reconfig)
# ======================================

def flujo_planificador_oco(entry_default: Optional[float] = None) -> None:
    """
    Flujo interactivo para planificar 2 OCOs con reconfig tras TP1,
    usando plan_dos_OCOs(ReconfParams) del Módulo 1.
    - Pide entry y USDT -> deriva qty
    - Pide split, TPs, SL global y parámetros de reconfiguración
    - Imprime precios clave y PnL de escenarios
    """
    print_title("Planificador OCO — 2 OCOs + reconfiguración")

    # Precio de entrada y cantidad (derivada desde USDT)
    entry_price = _ask_float("Precio de entrada", entry_default if entry_default else 4190.0)
    while entry_price <= 0:
        print("El precio de entrada debe ser > 0.")
        entry_price = _ask_float("Precio de entrada", 100.0)

    usdt = _ask_float("Monto invertido (USDT)", 621.2)
    while usdt < 0:
        print("El monto en USDT no puede ser negativo.")
        usdt = _ask_float("Monto invertido (USDT)", 621.2)

    qty = usdt / entry_price
    fee_side = FEE_SIDE

    print_section("Base de cálculo")
    print_kv("Entry", entry_price, 8)
    print_kv("USDT", usdt, 8)
    print_kv("qty = USDT / entry", qty, 8)
    print_kv("Fee por lado (%)", float(f"{fee_side*100:.3f}"))

    # Parámetros OCO
    print_section("Parámetros de los 2 OCO")
    split_oco1_pct = _ask_float("% qty al OCO1 (0-100)", 60.0)
    while split_oco1_pct < 0 or split_oco1_pct > 100:
        print("Debe estar entre 0 y 100.")
        split_oco1_pct = _ask_float("% qty al OCO1 (0-100)", 60.0)

    oco1_tp_pct    = _ask_float("TP del OCO1 (%) sobre entry", 1.2)
    oco2_tp_pct    = _ask_float("TP del OCO2 (%) sobre entry", 4.0)
    sl_pct_global  = _ask_float("SL global (%) bajo entry", 2.0)
    while sl_pct_global < 0:
        print("El SL global debe ser >= 0.")
        sl_pct_global = _ask_float("SL global (%) bajo entry", 2.0)

    print_section("Reconfiguración si TP1 se ejecuta")
    lock_after_tp1 = _ask_float("Tras TP1, subir SL2 a (%) SOBRE entry (0=BE, 0.2=+0.2%)", 0.2)
    tp2_retarget   = _ask_float("Tras TP1, nuevo TP2 (%) SOBRE entry", 5.0)

    # Construir plan
    reconf = ReconfParams(
        lock_profit_after_tp1_pct=lock_after_tp1,
        tp2_retarget_pct=tp2_retarget
    )

    plan = plan_dos_OCOs(
        entry=entry_price,
        qty_total=qty,
        fee_side=fee_side,
        sl_pct_global=sl_pct_global,
        split_qty_pct_for_oco1=split_oco1_pct,
        oco1_tp_pct=oco1_tp_pct,
        oco2_tp_pct=oco2_tp_pct,
        reconf=reconf
    )

    # Salida
    print_section("Resumen de órdenes")
    print_kv("Entry", plan["entrada"], 8)
    print_kv("SL global (precio)", plan["sl_price"], 8)

    print("\nOCO1:")
    for k in ("qty", "tp_%", "tp_price", "sl_%", "sl_price"):
        v = plan["oco1"][k]
        print_kv(f"  {k}", float(v) if isinstance(v, (int,float)) else v, 8 if "price" in k else None)

    print("\nOCO2:")
    for k in ("qty", "tp_%", "tp_price", "sl_%", "sl_price"):
        v = plan["oco2"][k]
        print_kv(f"  {k}", float(v) if isinstance(v, (int,float)) else v, 8 if "price" in k else None)

    print_section("Reconfiguración si TP1 ejecuta")
    rec = plan["reconfiguracion_si_TP1"]
    print_kv("Nuevo SL2 % sobre entry", float(rec["nuevo_SL2_%_sobre_entry"]))
    print_kv("Nuevo SL2 price", float(rec["nuevo_SL2_price"]), 8)
    print_kv("Nuevo TP2 % sobre entry", float(rec["nuevo_TP2_%_sobre_entry"]))
    print_kv("Nuevo TP2 price", float(rec["nuevo_TP2_price"]), 8)

    print_section("P&L por escenarios (neto)")
    for nombre, pnl in plan["pnl_escenarios"].items():
        print_kv(nombre, float(pnl), 8)


# ==============================
# Programa principal
# ==============================

def main():
    print_title("Herramienta Unificada de Niveles y Objetivos de PnL")

    datos = flujo_niveles()

    usar_entrada_alta = _ask_yes_no(
        "\n¿Usar el precio ENTRADA ALTA como precio de entrada por defecto para objetivos?",
        True,
    )
    entry_default = datos["EntradaAlta"] if usar_entrada_alta else None

    flujo_objetivos(entry_default)

    if _ask_yes_no("\n¿Quieres pasar al módulo de simulaciones TP/SL ahora?", True):
        flujo_simulaciones(entry_default)
    else:
        print("\nMódulo de simulaciones omitido.")

    if _ask_yes_no("\n¿Quieres evaluar R usando TP1/SL de tus niveles?", True):
        entry_eval = _ask_float("Entry para evaluar R", entry_default or datos["precio_actual"])
        usdt_eval  = _ask_float("USDT para evaluar R", 621.2)
        qty_eval   = usdt_eval / entry_eval
        evaluar_R(entry_eval, datos["niveles"].take_profit_1, datos["niveles"].stop_loss, qty_eval)

    # NUEVO: ofrecer OCO al final del flujo principal
    if _ask_yes_no("\n¿Quieres abrir el Planificador OCO ahora?", True):
        # Proponer ENTRADA ALTA como default para OCO
        entry_def_oco = datos["EntradaAlta"] if _ask_yes_no(
            "¿Usar ENTRADA ALTA como entry por defecto para OCO?", True
        ) else None
        flujo_planificador_oco(entry_def_oco)

    print("\nListo.\n")


if __name__ == "__main__":
    main()



           Herramienta Unificada de Niveles y Objetivos de PnL            

                       Niveles desde Medias Móviles                       


KeyboardInterrupt: Interrupted by user

In [ ]:
# ==============================
# Menú principal
# ==============================

from typing import Optional  # <-- por si no estaba

# Estado simple para reutilizar datos de 'Niveles'
_STATE = {"datos": None}

def _ask_int(msg: str, default: Optional[int] = None, min_v: Optional[int] = None, max_v: Optional[int] = None) -> int:
    tip = f" [{default}]" if default is not None else ""
    while True:
        s = input(f"{msg}{tip}: ").strip()
        if s == "" and default is not None:
            x = int(default)
        else:
            try:
                x = int(s)
            except ValueError:
                print("Valor inválido. Intenta de nuevo.")
                continue
        if (min_v is not None and x < min_v) or (max_v is not None and x > max_v):
            rango = []
            if min_v is not None: rango.append(str(min_v))
            if max_v is not None: rango.append(str(max_v))
            print(f"Elige un número entre {rango[0]} y {rango[-1]}.")
            continue
        return x

def menu():
    while True:
        print_title("Herramienta Unificada de Niveles y Objetivos de PnL — Menú")
        print("  1) Niveles desde Medias Móviles")
        print("  2) Objetivos de PnL (TP mínimo / SL máximo)")
        print("  3) Simulaciones TP/SL")
        print("  4) Evaluar R usando TP1/SL de tus niveles")
        print("  5) Planificador OCO (2 OCOs + reconfig)")  # <-- NUEVO
        print("  0) Salir")
        op = _ask_int("Selecciona opción", default=1, min_v=0, max_v=5)

        if op == 1:
            _STATE["datos"] = flujo_niveles()

        elif op == 2:
            entry_default = None
            datos = _STATE.get("datos")
            if datos is not None and _ask_yes_no("\n¿Usar ENTRADA ALTA como entry por defecto para objetivos?", True):
                entry_default = datos["EntradaAlta"]
            flujo_objetivos(entry_default)

        elif op == 3:
            entry_default = None
            datos = _STATE.get("datos")
            if datos is not None and _ask_yes_no("\n¿Usar ENTRADA ALTA como entry por defecto para simulaciones?", True):
                entry_default = datos["EntradaAlta"]
            flujo_simulaciones(entry_default)

        elif op == 4:
            datos = _STATE.get("datos")
            if datos is None:
                print("\nPrimero ejecuta 'Niveles' para obtener TP1/SL.\n")
                continue
            entry_eval = _ask_float("Entry para evaluar R", datos["precio_actual"])
            usdt_eval  = _ask_float("USDT para evaluar R", 621.2)
            qty_eval   = usdt_eval / entry_eval
            evaluar_R(entry_eval, datos["niveles"].take_profit_1, datos["niveles"].stop_loss, qty_eval)

        elif op == 5:  # <-- NUEVO
            datos = _STATE.get("datos")
            entry_default = None
            if datos is not None and _ask_yes_no("\n¿Usar ENTRADA ALTA como entry por defecto para OCO?", True):
                entry_default = datos["EntradaAlta"]
            flujo_planificador_oco(entry_default)

        elif op == 0:
            print("\nListo.\n")
            break


def main():
    menu()


if __name__ == "__main__":
    main()


        Herramienta Unificada de Niveles y Objetivos de PnL — Menú        
  1) Niveles desde Medias Móviles
  2) Objetivos de PnL (TP mínimo / SL máximo)
  3) Simulaciones TP/SL
  4) Evaluar R usando TP1/SL de tus niveles
  5) Planificador OCO (2 OCOs + reconfig)
  0) Salir

                       Niveles desde Medias Móviles                       

--------------------------------------------------------------------------
                                Parámetros                                
--------------------------------------------------------------------------
  MA(7)                                            200,34
  MA(25)                                           200,98
  MA(99)                                           194,11
  Precio actual                                    197,40
  Máximo reciente (TP1 base)                       204,96

--------------------------------------------------------------------------
                            Niveles sugeridos          